In [44]:
import pandas as pd

from bs4 import BeautifulSoup

from tqdm import tqdm
tqdm.pandas()

In [2]:
SRC_DATA = '../../data/data_clean/polisumm_final.csv'

In [3]:
data = pd.read_csv(SRC_DATA)

# Rescraping Expert Opinions

In [7]:
ARTICLE_SOURCE = '../../data/webpage_data/articles/'

In [8]:
data['article_file'] = ARTICLE_SOURCE + data['title'].str.replace('?', '') + '.html'

C:\Users\ndeas\miniconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [65]:
def get_expert_opinions(article_src):
    article_src = article_src[:article_src.find(':')] if ':' in article_src else article_src
    with open(article_src, 'r', encoding = 'utf8') as f:
        src_html = f.read()
        
    bs = BeautifulSoup(src_html)
    
    right_experts = bs.find_all('div', {'class': lambda e: ('paragraph-6' in e and 'right' in e and 'bullet' in e and not ('w-dyn-bind-empty' in e)) if e else False})
    right_experts = [e.p.text for e in right_experts if e.p]
    
    left_experts = bs.find_all('div', {'class': lambda e: ('paragraph-6' in e and 'left' in e and 'bullet' in e and not ('w-dyn-bind-empty' in e)) if e else False})
    left_experts = [e.p.text for e in left_experts if e.p]
    
    right_experts = [e[:-1 * e[::-1].find('”')] for e in right_experts]
    left_experts = [e[:-1 * e[::-1].find('”')] for e in left_experts]
    
    right_experts = [e.replace('”', '').replace('“', '') for e in right_experts]
    left_experts = [e.replace('”', '').replace('“', '') for e in left_experts]
    
    return left_experts, right_experts

In [66]:
expert_ops = data['article_file'].progress_apply(get_expert_opinions)

100%|██████████████████████████████████████████████████████████████████████████████| 1199/1199 [00:12<00:00, 98.63it/s]


In [69]:
left_ops = ['|'.join(tup[0]) for tup in expert_ops]
right_ops = ['|'.join(tup[1]) for tup in expert_ops]

In [70]:
data['left_op'] = left_ops
data['right_op'] = right_ops

In [72]:
data = data.drop('article_file', axis = 1)

In [73]:
data.to_csv(SRC_DATA)